In [1]:
%matplotlib inline
import tensorflow as tf 
import matplotlib.pyplot as plt
import numpy as np

# Pravimo feature matricu od trening podataka
def create_feature_matrix(x, nb_features):
  tmp_features = []
  for deg in range(1, nb_features+1):
    tmp_features.append(np.power(x, deg))
  return np.column_stack(tmp_features)
  

# Preciznost
np.set_printoptions(suppress=True, precision=15)  

# Učitavanje podataka
filename = 'funky.csv'
all_data = np.loadtxt(filename, delimiter=',', skiprows=0, usecols=(0, 1))
data = dict()
featured_data = dict()
data['x'] = all_data[:, 0]
data['y'] = all_data[:, 1]
nb_samples = data['x'].shape[0]

# Normalizacija
data['x'] = (data['x'] - np.mean(data['x'])) / np.std(data['x'])
data['y'] = (data['y'] - np.mean(data['y'])) / np.std(data['y'])

# Iscrtavanje podataka
plt.scatter(data['x'], data['y'])
plt.xlabel('X value')
plt.ylabel('Y value')

colors = ['r','m','orange','yellow','blue','g']
c = 0
all_losses = []

for nb_features in range(1, 7):   
  
  featured_data = create_feature_matrix(data['x'], nb_features)
  epoch_loss_list = []
  
  # Definisanje modela
  X = tf.placeholder(shape=(None, nb_features), dtype=tf.float32)
  Y = tf.placeholder(shape=(None), dtype=tf.float32)
  w = tf.Variable(tf.zeros(nb_features))
  bias = tf.Variable(0.0)

  w_col = tf.reshape(w, (nb_features, 1))
  hyp = tf.add(tf.matmul(X, w_col), bias)
  
  # Funkcija troska i Adam optimizator
  Y_col = tf.reshape(Y, (-1, 1))
  loss = tf.reduce_mean(tf.square(hyp - Y_col))
  opt_op = tf.train.AdamOptimizer().minimize(loss)
 
  # Treniranje
  with tf.Session() as sess:
    print("Traning with polynomial degree = {}".format(nb_features))
    sess.run(tf.global_variables_initializer())
    nb_epochs = 100 # Realno i ovo je mnogo

    # Jedna epoha = jedan prolaz kroz ceo trening set
    for epoch in range(nb_epochs):
      epoch_loss = 0

      for sample in range(nb_samples):
        feed = {X: featured_data[sample].reshape((1, nb_features)), 
                Y: data['y'][sample]}
        _, curr_loss = sess.run([opt_op, loss], feed_dict=feed)
        epoch_loss += curr_loss

      epoch_loss /= nb_samples
      epoch_loss_list.append(epoch_loss)
      if (epoch + 1) % 20 == 0:
        print('Epoch: {}/{}| Avg loss: {:.5f}'.format(epoch + 1, nb_epochs, epoch_loss))

    # Završili smo treniranje (100 puta prošli kroz ceo trening set)  
    all_losses.append(epoch_loss_list)
    w_val = sess.run(w)
    bias_val = sess.run(bias)
    xs = create_feature_matrix(np.linspace(-2, 4, 100), nb_features)
    hyp_val = sess.run(hyp, feed_dict={X: xs})
    
    plt.plot(xs[:, 0].tolist(), hyp_val.tolist(), color=colors[c], linewidth=3, label=str(nb_features))
    c += 1
    
    print("---------------------------------")


plt.title('All 6 regression hypothesis', fontsize=19)
plt.legend(loc='lower right')
plt.xlim([-2, 4])
plt.ylim([-3, 4])
plt.show()

c = 0
plt.title('Loss function & polynomial degree', fontsize=19)
plt.xlabel('Epoch number')
plt.ylabel('Loss function value')

for loss in all_losses:
  x_axis = list(range(nb_epochs))
  y_axis = loss
  plt.plot(x_axis, y_axis, linewidth=3, color=colors[c], label=str(c + 1))
  c += 1
  
plt.legend(loc='upper right')
plt.show()


'''
Možemo primetiti da je nemoguće rešiti ovakav dataset sa stepenom 1 ili 2. 
Polinomi stepena 3 ili više rešavaju problem ali izgleda da je sve preko 3 nepotrebno.
Na grafiku možemo videti da se regresione krive za stepene 3, 4, 5, 6 skoro u potpunosti
poklapaju, a i prosečni loss je sličan. 
'''









OSError: ignored